In [1]:
import pandas as pd
from osgeo import gdal
import math
import os
import glob
from PIL import Image # PIL library supports only TIF format but not newer TIFF
import numpy as np
import shutil

In [2]:
data_directory = '../data/'
asf_data_directory = data_directory + '3_processed_ASF_data/'
interpolated_AIS_data_directory = data_directory + '5_interpolated_AIS_data/'
#annotated_asf_data_directory = data_directory + '6_annotated_ASF_data/'
cutted_asf_data_directory = data_directory + '6_cutted_ASF_data/'
final_dataset = data_directory + '7_final_dataset/'
final_dataset_images = final_dataset + 'images/'
final_dataset_labels = final_dataset + 'labels/'
###
second_final_dataset = data_directory + '8_final_dataset_second_version/'
second_test = second_final_dataset + 'test/'
second_train = second_final_dataset + 'train/'
second_val = second_final_dataset + 'val/'
#sliced_asf_data_directory = data_directory +'7_sliced_ASF_data/'
results_directory = '../results/'

### 1. Create train/test split

In [4]:
stopfiles = ['.gitkeep', '.ipynb_checkpoints', 'original_picture.txt', 'original_picture.png.aux.xml', 'original_picture.png']
TEST_RATIO = 0.3
VALIDATION_RATIO = 0.3

pictures=list()
for file in os.listdir(cutted_asf_data_directory):
    if not file in stopfiles:
        pictures.append(file)

display(pictures)
total_number_of_pictures = len(pictures)
print(f"total number_of_folder_pictures: {total_number_of_pictures}")
print()
nr_test_pictures = round(TEST_RATIO*total_number_of_pictures)
print(f"number_of_test_folder_pictures: {nr_test_pictures}")
nr_train_pictures = total_number_of_pictures - nr_test_pictures
print(f"number_of_train_folder_pictures: {nr_train_pictures}")
print()
nr_val_pictures = round(VALIDATION_RATIO*nr_train_pictures)
print(f"nr_val_folder_pictures: {nr_val_pictures}")

['subset_0_of_S1A_IW_GRDH_1SDV_20210106T015004_20210106T015033_036011_04381A_71E9_TC',
 'subset_10_of_S1A_IW_GRDH_1SDV_20210307T015003_20210307T015032_036886_045698_BCE0_TC',
 'subset_11_of_S1B_IW_GRDH_1SDV_20210306T015804_20210306T015829_025888_031671_3DA0_TC',
 'subset_12_of_S1B_IW_GRDH_1SDV_20210406T014917_20210406T014942_026340_0324E7_7951_TC',
 'subset_13_of_S1B_IW_GRDH_1SDV_20210505T015737_20210505T015806_026763_03326A_27A5_TC',
 'subset_14_of_S1B_IW_GRDH_1SDV_20210505T015806_20210505T015831_026763_03326A_4A6B_TC',
 'subset_15_of_S1B_IW_GRDH_1SDV_20210605T014920_20210605T014945_027215_03403B_1C08_TC',
 'subset_16_of_S1B_IW_GRDH_1SDV_20210605T135153_20210605T135218_027222_034079_67E0_TC',
 'subset_17_of_S1B_IW_GRDH_1SDV_20210605T135218_20210605T135243_027222_034079_5356_TC',
 'subset_18_of_S1B_IW_GRDH_1SDV_20210105T015737_20210105T015806_025013_02FA23_627F_TC',
 'subset_19_of_S1B_IW_GRDH_1SDV_20210105T015806_20210105T015831_025013_02FA23_C4CD_TC',
 'subset_1_of_S1A_IW_GRDH_1SDV_20

total number_of_folder_pictures: 21

number_of_test_folder_pictures: 6
number_of_train_folder_pictures: 15

nr_val_folder_pictures: 4


In [5]:
test_folders = pictures[0:nr_test_pictures]
print(f"test_folders: {test_folders}")
print()
train_folders = pictures[nr_test_pictures:]
print(f"train_folders: {train_folders}")
print()
val_folders = pictures[nr_test_pictures:nr_test_pictures+nr_val_pictures]
print(f"val_folders: {val_folders}")

test_folders: ['subset_0_of_S1A_IW_GRDH_1SDV_20210106T015004_20210106T015033_036011_04381A_71E9_TC', 'subset_10_of_S1A_IW_GRDH_1SDV_20210307T015003_20210307T015032_036886_045698_BCE0_TC', 'subset_11_of_S1B_IW_GRDH_1SDV_20210306T015804_20210306T015829_025888_031671_3DA0_TC', 'subset_12_of_S1B_IW_GRDH_1SDV_20210406T014917_20210406T014942_026340_0324E7_7951_TC', 'subset_13_of_S1B_IW_GRDH_1SDV_20210505T015737_20210505T015806_026763_03326A_27A5_TC', 'subset_14_of_S1B_IW_GRDH_1SDV_20210505T015806_20210505T015831_026763_03326A_4A6B_TC']

train_folders: ['subset_15_of_S1B_IW_GRDH_1SDV_20210605T014920_20210605T014945_027215_03403B_1C08_TC', 'subset_16_of_S1B_IW_GRDH_1SDV_20210605T135153_20210605T135218_027222_034079_67E0_TC', 'subset_17_of_S1B_IW_GRDH_1SDV_20210605T135218_20210605T135243_027222_034079_5356_TC', 'subset_18_of_S1B_IW_GRDH_1SDV_20210105T015737_20210105T015806_025013_02FA23_627F_TC', 'subset_19_of_S1B_IW_GRDH_1SDV_20210105T015806_20210105T015831_025013_02FA23_C4CD_TC', 'subset_1_of

### 2. Copy from folders only pictures which has corresponding txt file

In [11]:
FORMAT_TO_COPY = 'png'

In [12]:
def copy_pictures_and_labels(source_path, source_images_folders, destination_path_images, destination_path_labels, destination_format, counter):
    source_path = source_path
    source_images_folders = source_images_folders
    destination_path_images = destination_path_images
    destination_path_labels = destination_path_labels
    destination_format = destination_format
    image_counter = counter
    labels_counter = counter
    print(destination_path_labels)

    for folder in source_images_folders:
        #print()
        #print(folder)
        # choose only files with txt annotations
        #print(os.listdir(source_path + folder))
        #print()
        labels=list()
        images=list()
        for file in os.listdir(source_path + folder):
            if file.endswith(".txt"):
                labels.append(file)
                image_file = file.replace(".txt", ".png")
                images.append(image_file)
        #print(f"images: {images}")
        #print(f"labels: {labels}")
        # labels
        for file in labels:
            src = source_path + '/' + folder + '/' + file
            #print(f"src: {src}")
            dst = destination_path_labels + str(labels_counter) + '.txt' #folder + file
            #print(f"dst: {dst}")
            shutil.copy(src, dst)
            labels_counter += 1
        # pictures
        counter = counter
        for file in images:
            src = source_path + '/' + folder + '/' + file
            #print(f"src: {src}")
            dst = destination_path_images + str(image_counter) + '.' + destination_format #folder + file
            #print(f"dst: {dst}")
            shutil.copy(src, dst)
            image_counter += 1
            
    return image_counter

## First dataset creation option

2.0 Optionally - delete the contents of the files

In [7]:
files = glob.glob(f'{final_dataset}*/*/*.jpg')
for f in files:
    os.remove(f)
files = glob.glob(f'{final_dataset}*/*/*.tif')
for f in files:
    os.remove(f)
files = glob.glob(f'{final_dataset}*/*/*.txt')
for f in files:
    os.remove(f)

2.1 Test pictures

In [8]:
copy_pictures_and_labels(source_path = cutted_asf_data_directory,
                        source_images_folders = test_folders,
                        destination_path_images = final_dataset_images + 'test/',
                        destination_path_labels = final_dataset_labels + 'test/',
                        destination_format=FORMAT_TO_COPY)

../data/7_final_dataset/labels/test/

S1A_IW_GRDH_1SDV_20210204T015812_20210204T015841_036434_0446E1_2157_TC
['subset1.jpg', 'subset1.jpg.aux.xml', 'subset10.jpg', 'subset10.jpg.aux.xml', 'subset100.jpg', 'subset100.jpg.aux.xml', 'subset101.jpg', 'subset101.jpg.aux.xml', 'subset101.txt', 'subset102.jpg', 'subset102.jpg.aux.xml', 'subset103.jpg', 'subset103.jpg.aux.xml', 'subset104.jpg', 'subset104.jpg.aux.xml', 'subset104.tif', 'subset105.jpg', 'subset105.jpg.aux.xml', 'subset106.jpg', 'subset106.jpg.aux.xml', 'subset107.jpg', 'subset107.jpg.aux.xml', 'subset108.jpg', 'subset108.jpg.aux.xml', 'subset109.jpg', 'subset109.jpg.aux.xml', 'subset11.jpg', 'subset11.jpg.aux.xml', 'subset110.jpg', 'subset110.jpg.aux.xml', 'subset111.jpg', 'subset111.jpg.aux.xml', 'subset112.jpg', 'subset112.jpg.aux.xml', 'subset113.jpg', 'subset113.jpg.aux.xml', 'subset114.jpg', 'subset114.jpg.aux.xml', 'subset114.txt', 'subset115.jpg', 'subset115.jpg.aux.xml', 'subset116.jpg', 'subset116.jpg.aux.xml', 'subset

FileNotFoundError: [Errno 2] No such file or directory: '../data/6_cutted_ASF_data//S1A_IW_GRDH_1SDV_20210204T015812_20210204T015841_036434_0446E1_2157_TC/subset101.tif'

2.2 Train pictures

In [34]:
copy_pictures_and_labels(source_path = cutted_asf_data_directory,
                        source_images_folders = train_folders,
                        destination_path_images = final_dataset_images + 'train/',
                        destination_path_labels = final_dataset_labels + 'train/',
                        destination_format=FORMAT_TO_COPY)

../data/7_final_dataset/labels/train/

S1B_IW_GRDH_1SDV_20210105T015737_20210105T015806_025013_02FA23_627F_TC
['subset1.tif', 'subset10.tif', 'subset100.tif', 'subset100.txt', 'subset101.tif', 'subset101.txt', 'subset102.tif', 'subset102.txt', 'subset103.tif', 'subset104.tif', 'subset105.tif', 'subset106.tif', 'subset107.tif', 'subset108.tif', 'subset109.tif', 'subset11.tif', 'subset110.tif', 'subset111.tif', 'subset112.tif', 'subset112.txt', 'subset113.tif', 'subset114.tif', 'subset114.txt', 'subset115.tif', 'subset116.tif', 'subset117.tif', 'subset118.tif', 'subset119.tif', 'subset12.tif', 'subset120.tif', 'subset121.tif', 'subset122.tif', 'subset123.tif', 'subset124.tif', 'subset125.tif', 'subset126.tif', 'subset127.tif', 'subset128.tif', 'subset129.tif', 'subset13.tif', 'subset130.tif', 'subset131.tif', 'subset132.tif', 'subset14.tif', 'subset15.tif', 'subset16.tif', 'subset17.tif', 'subset18.tif', 'subset19.tif', 'subset2.tif', 'subset20.tif', 'subset21.tif', 'subset22.tif', 'subs

2.3 Validation pictures

In [35]:
copy_pictures_and_labels(source_path = cutted_asf_data_directory,
                        source_images_folders = val_folders,
                        destination_path_images = final_dataset_images + 'val/',
                        destination_path_labels = final_dataset_labels + 'val/',
                        destination_format=FORMAT_TO_COPY)

../data/7_final_dataset/labels/val/

S1B_IW_GRDH_1SDV_20210105T015737_20210105T015806_025013_02FA23_627F_TC
['subset1.tif', 'subset10.tif', 'subset100.tif', 'subset100.txt', 'subset101.tif', 'subset101.txt', 'subset102.tif', 'subset102.txt', 'subset103.tif', 'subset104.tif', 'subset105.tif', 'subset106.tif', 'subset107.tif', 'subset108.tif', 'subset109.tif', 'subset11.tif', 'subset110.tif', 'subset111.tif', 'subset112.tif', 'subset112.txt', 'subset113.tif', 'subset114.tif', 'subset114.txt', 'subset115.tif', 'subset116.tif', 'subset117.tif', 'subset118.tif', 'subset119.tif', 'subset12.tif', 'subset120.tif', 'subset121.tif', 'subset122.tif', 'subset123.tif', 'subset124.tif', 'subset125.tif', 'subset126.tif', 'subset127.tif', 'subset128.tif', 'subset129.tif', 'subset13.tif', 'subset130.tif', 'subset131.tif', 'subset132.tif', 'subset14.tif', 'subset15.tif', 'subset16.tif', 'subset17.tif', 'subset18.tif', 'subset19.tif', 'subset2.tif', 'subset20.tif', 'subset21.tif', 'subset22.tif', 'subset

# 3. Second version of dataset creation

In [13]:
# clear file structure
files = glob.glob(f'{second_final_dataset}*/*/*.jpg')
for f in files:
    os.remove(f)
files = glob.glob(f'{second_final_dataset}*/*/*.tif')
for f in files:
    os.remove(f)
files = glob.glob(f'{second_final_dataset}*/*/*.txt')
for f in files:
    os.remove(f)
    
# start counting
counter = 0
# TEST PICTURES
counter = copy_pictures_and_labels(source_path = cutted_asf_data_directory,
                                    source_images_folders = test_folders,
                                    destination_path_images = second_test + 'images/',
                                    destination_path_labels = second_test + 'labels/',
                                    destination_format=FORMAT_TO_COPY,
                                    counter = counter)
print(f"counter: {counter}")
# TRAIN PICTURES
counter = copy_pictures_and_labels(source_path = cutted_asf_data_directory,
                                    source_images_folders = train_folders,
                                    destination_path_images = second_train + 'images/',
                                    destination_path_labels = second_train + 'labels/',
                                    destination_format=FORMAT_TO_COPY,
                                    counter = counter)
print(f"counter: {counter}")
counter = copy_pictures_and_labels(source_path = cutted_asf_data_directory,
                                    source_images_folders = val_folders,
                                    destination_path_images = second_val + 'images/',
                                    destination_path_labels = second_val + 'labels/',
                                    destination_format=FORMAT_TO_COPY,
                                    counter = counter)
print(f"counter: {counter}")

../data/8_final_dataset_second_version/test/labels/
counter: 19
../data/8_final_dataset_second_version/train/labels/
counter: 114
../data/8_final_dataset_second_version/val/labels/
counter: 129


# OLD Second version of dataset creation

2.0 Optionally - delete the contents of the files

In [59]:
files = glob.glob(f'{second_final_dataset}*/*/*.jpg')
for f in files:
    os.remove(f)
files = glob.glob(f'{second_final_dataset}*/*/*.tif')
for f in files:
    os.remove(f)
files = glob.glob(f'{second_final_dataset}*/*/*.txt')
for f in files:
    os.remove(f)

In [43]:
counter = 0

2.1 Test pictures

In [60]:
copy_pictures_and_labels(source_path = cutted_asf_data_directory,
                        source_images_folders = test_folders,
                        destination_path_images = second_test + 'images/',
                        destination_path_labels = second_test + 'labels/',
                        destination_format=FORMAT_TO_COPY,
                        counter = counter)

../data/8_final_dataset_second_version/test/labels/

subset_0_of_S1A_IW_GRDH_1SDV_20210106T015004_20210106T015033_036011_04381A_71E9_TC

images: []
labels: []

subset_10_of_S1A_IW_GRDH_1SDV_20210307T015003_20210307T015032_036886_045698_BCE0_TC

images: ['subset115.jpg', 'subset18.jpg', 'subset25.jpg', 'subset28.jpg', 'subset29.jpg', 'subset35.jpg', 'subset37.jpg', 'subset38.jpg', 'subset42.jpg', 'subset46.jpg', 'subset47.jpg', 'subset48.jpg', 'subset51.jpg', 'subset55.jpg', 'subset56.jpg', 'subset57.jpg', 'subset64.jpg', 'subset65.jpg', 'subset66.jpg', 'subset67.jpg', 'subset71.jpg', 'subset76.jpg', 'subset84.jpg', 'subset93.jpg', 'subset94.jpg']
labels: ['subset115.txt', 'subset18.txt', 'subset25.txt', 'subset28.txt', 'subset29.txt', 'subset35.txt', 'subset37.txt', 'subset38.txt', 'subset42.txt', 'subset46.txt', 'subset47.txt', 'subset48.txt', 'subset51.txt', 'subset55.txt', 'subset56.txt', 'subset57.txt', 'subset64.txt', 'subset65.txt', 'subset66.txt', 'subset67.txt', 'subset71.txt',

2.2 Train pictures

In [38]:
copy_pictures_and_labels(source_path = cutted_asf_data_directory,
                        source_images_folders = train_folders,
                        destination_path_images = second_train + 'images/',
                        destination_path_labels = second_train + 'labels/',
                        destination_format=FORMAT_TO_COPY)

../data/8_final_dataset_second_version/train/labels/

subset_15_of_S1B_IW_GRDH_1SDV_20210605T014920_20210605T014945_027215_03403B_1C08_TC

images: []
labels: []

subset_16_of_S1B_IW_GRDH_1SDV_20210605T135153_20210605T135218_027222_034079_67E0_TC

images: []
labels: []

subset_17_of_S1B_IW_GRDH_1SDV_20210605T135218_20210605T135243_027222_034079_5356_TC

images: ['subset16.jpg', 'subset50.jpg', 'subset57.jpg', 'subset70.jpg', 'subset80.jpg', 'subset81.jpg']
labels: ['subset16.txt', 'subset50.txt', 'subset57.txt', 'subset70.txt', 'subset80.txt', 'subset81.txt']

subset_18_of_S1B_IW_GRDH_1SDV_20210105T015737_20210105T015806_025013_02FA23_627F_TC

images: ['subset21.jpg', 'subset24.jpg', 'subset27.jpg', 'subset31.jpg', 'subset41.jpg', 'subset42.jpg', 'subset51.jpg', 'subset52.jpg', 'subset61.jpg', 'subset62.jpg', 'subset63.jpg', 'subset71.jpg', 'subset72.jpg', 'subset73.jpg', 'subset8.jpg']
labels: ['subset21.txt', 'subset24.txt', 'subset27.txt', 'subset31.txt', 'subset41.txt', 'subset42.tx

2.3 Validation pictures

In [39]:
copy_pictures_and_labels(source_path = cutted_asf_data_directory,
                        source_images_folders = val_folders,
                        destination_path_images = second_val + 'images/',
                        destination_path_labels = second_val + 'labels/',
                        destination_format=FORMAT_TO_COPY)

../data/8_final_dataset_second_version/val/labels/

subset_15_of_S1B_IW_GRDH_1SDV_20210605T014920_20210605T014945_027215_03403B_1C08_TC

images: []
labels: []

subset_16_of_S1B_IW_GRDH_1SDV_20210605T135153_20210605T135218_027222_034079_67E0_TC

images: []
labels: []

subset_17_of_S1B_IW_GRDH_1SDV_20210605T135218_20210605T135243_027222_034079_5356_TC

images: ['subset16.jpg', 'subset50.jpg', 'subset57.jpg', 'subset70.jpg', 'subset80.jpg', 'subset81.jpg']
labels: ['subset16.txt', 'subset50.txt', 'subset57.txt', 'subset70.txt', 'subset80.txt', 'subset81.txt']

subset_18_of_S1B_IW_GRDH_1SDV_20210105T015737_20210105T015806_025013_02FA23_627F_TC

images: ['subset21.jpg', 'subset24.jpg', 'subset27.jpg', 'subset31.jpg', 'subset41.jpg', 'subset42.jpg', 'subset51.jpg', 'subset52.jpg', 'subset61.jpg', 'subset62.jpg', 'subset63.jpg', 'subset71.jpg', 'subset72.jpg', 'subset73.jpg', 'subset8.jpg']
labels: ['subset21.txt', 'subset24.txt', 'subset27.txt', 'subset31.txt', 'subset41.txt', 'subset42.txt'

# OLD

In [65]:
source_path = cutted_asf_data_directory
source_images_folders = test_folders
destination_path_images = final_dataset_images + 'test/'
destination_path_labels = final_dataset_labels + 'test/'
print(destination_path_labels)

for folder in source_images_folders:
    print()
    print(folder)
    # choose only files with txt annotations
    print(os.listdir(source_path + folder))
    print()
    labels=list()
    images=list()
    for file in os.listdir(source_path + folder):
        if file.endswith(".txt"):
            labels.append(file)
            image_file = file.replace(".txt", ".tif")
            images.append(image_file)
    # copy files - rename them while copying
    # labels
    for file in labels:
        src = source_path + '/' + folder + '/' + file
        dst = destination_path_labels + folder + file
        shutil.copy(src, dst)
    # pictures
    for file in images:
        src = source_path + '/' + folder + '/' + file
        dst = destination_path_images + folder + file
        shutil.copy(src, dst)
    

print(labels)
print()
print(images)

../data/7_final_dataset/labels/test/
S1A_IW_GRDH_1SDV_20210204T015812_20210204T015841_036434_0446E1_2157_TC
['subset1.tif', 'subset1.txt', 'subset10.tif', 'subset10.txt', 'subset11.tif', 'subset12.tif', 'subset2.tif', 'subset3.tif', 'subset3.txt', 'subset4.tif', 'subset4.txt', 'subset5.tif', 'subset5.txt', 'subset6.tif', 'subset6.txt', 'subset7.tif', 'subset8.tif', 'subset9.tif', 'subset9.txt']

['subset1.txt', 'subset10.txt', 'subset3.txt', 'subset4.txt', 'subset5.txt', 'subset6.txt', 'subset9.txt']

['subset1.tif', 'subset10.tif', 'subset3.tif', 'subset4.tif', 'subset5.tif', 'subset6.tif', 'subset9.tif']
